In [1]:
using HCubature

## HCubature

Steven G. Johnson has written a simple C package for adaptive multidimensional integration (cubature) of vector-valued integrands over hypercubes, i.e. to compute integrals of the form:
$$\int_{a_1}^{b_1}\int_{a_2}^{b_2}\cdots\int_{a_n}^{b_n} \vec{f}(\vec{x}) d^n\vec{x}$$
(Of course, it can handle scalar integrands as the special case where $\vec{f}$ is a one-dimensional vector: the dimensionalities of $\vec{f}$ and $\vec{x}$ are independent.) The integrand can be evaluated for an array of points at once to enable easy parallelization. The code, which is distributed as free software under the terms of the GNU General Public License (v2 or later), implements two algorithms for adaptive integration.

The first, h-adaptive integration (recursively partitioning the integration domain into smaller subdomains, applying the same integration rule to each, until convergence is achieved), is based on the algorithms described in:

A. C. Genz and A. A. Malik, "An adaptive algorithm for numeric integration over an N-dimensional rectangular region," J. Comput. Appl. Math. 6 (4), 295–302 (1980).

J. Berntsen, T. O. Espelid, and A. Genz, "An adaptive algorithm for the approximate calculation of multiple integrals," ACM Trans. Math. Soft. 17 (4), 437–451 (1991).

This algorithm is best suited for a moderate number of dimensions (say, < 7), and is superseded for high-dimensional integrals by other methods (e.g. Monte Carlo variants or sparse grids).

(Note that we do not use any of the original DCUHRE code by Genz, which is not under a free/open-source license.) Our code is based in part on code borrowed from the HIntLib numeric-integration library by Rudolf Schürer and from code for Gauss-Kronrod quadrature (for 1d integrals) from the GNU Scientific Library, both of which are free software under the GNU GPL. (Another free-software multi-dimensional integration library, unrelated to our code here but also implementing the Genz–Malik algorithm among other techniques, is Cuba.)

The second, p-adaptive integration (repeatedly doubling the degree of the quadrature rules until convergence is achieved), is based on a tensor product of Clenshaw–Curtis quadrature rules. This algorithm is often superior to h-adaptive integration for smooth integrands in a few (≤3) dimensions, but is a poor choice in higher dimensions or for non-smooth integrands.

For the most part, the p-adaptive routines below are drop-in replacements for the h-adaptive routines, with the same arguments etcetera, so you can experiment to see which one works best for your problem. One difference: the h-adaptive routines do *not* evaluate the integrand on the boundaries of the integration volume, whereas the p-adaptive routines *do* evaluate the integrand at the boundaries. This means that the p adaptive routines require more care in cases where there are singularities at the boundaries.
I am also grateful to Dmitry Turbiner (dturbiner ατ alum.mit.edu), who implemented an initial prototype of the "vectorized" functionality (see below) for evaluating an array of points in a single call, which facilitates parallelization of the integrand evaluation.

Integration is performed on the n-dimensional unit hypercube $[0, 1]^n$. If you want to compute an integral over a different set, you have to scale the integrand function in order to have an equivalent integral on $[0, 1]^n$. For example, recall that in one dimension

$$\int_a^b f(x)\,dx \rightarrow \int_0^1 f(a + y(b-a)) (b-a)\,dy$$
where the final $(b - a)$ is the one-dimensional version of the Jacobian. This generalizes straightforwardly to more than one dimension.

### Example

To evaluate the integral $$\int_1^3 x^2\,dx$$ we have to make a change of variables to scale the domain of integration to the interval $[0, 1]$. We can do this by making the substitutions

* $x\rightarrow 1 + 2x$
* $dx\rightarrow 2\,dx$

Then we have $$\int_1^3 x^2\,dx \rightarrow \int_0^1 (1 + 2x)^2 (2\,dx)$$

### Example

Evaluate the integral $$\int_{y=0}^2\int_{x=0}^1 x+y\,dx\,dy$$

In [6]:
# Note: x is a vector with components x[1], x[2], ..., x[n]
# the arguments after the function definintion of f: x -> f(x) are
# the opposite corners of an n-dimensional cube showing the intervals
# over which each component are integrated. For this example, the
# arguments [0,0] and [1,2] indicate that x[1] is integrated over
# the interval [0, 1] and x[2] is integrated over the interval [0, 2]

hcubature(x -> x[1]+x[2], [0,0], [1,2])

(3.0, 4.440892098500626e-16)

### Example

Evaluate the integral $$\iiint_E dV = \int_{z=0}^1\int_{y=0}^{1}\int_{x=0}^{1} 2(2-2z)(4-(2y-2yz)^2)\,dx\,dy\,dz$$

In [9]:
hcubature(x -> (4 - 4*x[3])*(4 - (2*x[2] - 2*x[2]*x[3])^2), [0,0,0], [1,1,1])

(6.666666666666668, 0.0)

### Example

Evaluate the integral $$\int_0^1\int_0^1\int_0^1 2\pi^3 y z^2\cos(\pi yz(2x - 1))\sin(\pi yz)e^{\pi z}\,dx\,dy\,dz$$

In [4]:
function f(x)
    2*pi^3 * x[2] * x[3]^2 * cos(pi*x[2]*x[3]*(2*x[1] - 1))*sin(pi*x[2]*x[3])*exp(pi*x[3])
end

f (generic function with 1 method)

In [6]:
hcubature(f, [0,0,0], [1,1,1], atol=1e-6, maxevals=10^8)

(54.98607586777753, 9.997989776698802e-7)

### Example

Evaluate $\iint_E 15z\,dV$ where $E$ is the region between $2x + y + z = 4$ and $4x + 4y + 2z = 20$ that is in front of the region in the $yz$-plane bounded by $z = 2y^2$ and $z = \sqrt{4y}$.

As an iterated integral we need to evaluate $$\int_{y=0}^1\int_{z=2y^2}^{\sqrt{4y}}\int_{x=2-y/2-z/2}^{5-y-z/2} 15z\,dx\,dz\,dy$$

To use the 'hcubature' function, we need to make transformations (substitutions) to make the limits of integration between constant values.

To start, make the substitution

* $z \rightarrow 2y^2 + z(\sqrt{4y} - 2y^2) = 2y^2 + z\sqrt{4y} - 2zy^2$
* $dz \rightarrow (\sqrt{4y} - 2y^2)\,dz$

This results in the integral $$\int_{y=0}^1\int_{z=0}^1\int_{x=2-y/2-(2y^2 + z\sqrt{4y}-2zy^2)/2}^{5-y-(2y^2 + z\sqrt{4y}-2zy^2)/2} 15(2y^2 + z\sqrt{4y} - 2zy^2)(\sqrt{4y} - 2y^2)\,dx\,dz\,dy$$

Next make the substitutions

* $x\rightarrow 2-y/2-(2y^2+z\sqrt{4y}-2zy^2)/2 + x(3-y/2)$
* $dx\rightarrow (3-y/2)\,dx$

This results in the integral $$\int_{y=0}^1\int_{z=0}^1\int_{x=0}^1 15(2y^2 + z\sqrt{4y} - 2zy^2)(3 - \tfrac{1}{2}y)(\sqrt{4y} - 2y^2)\,dx\,dz\,dy$$

In [7]:
function f(x)
    15*(2*x[2]^2 + x[3]*sqrt(4*x[2]) - 2*x[3]*x[2]^2)*(3 - x[2]/2)*(sqrt(4*x[2]) - 2*x[2]^2)
end

f (generic function with 1 method)

In [8]:
hcubature(f, [0,0,0], [1,1,1])

(24.499999999999996, 1.0658141036401503e-14)

### Example

Evaluate $\iint_D 1\,dA$, where $D$ is a square of side length 2 with one corner at the origin and the opposite corner at the point $(2,2)$ using polar coordinates.

To do this in polar coordinates we will have to use two integrals since the outer boundary of integration changes at $\theta = \pi/4$.

In polar coordinates we have $$\iint_D 1\,dA = \int_{\theta=0}^{\pi/4}\int_{r=0}^{2\sec\theta} 1r\,dr\,d\theta +
\int_{\theta=\pi/4}^{\pi/2}\int_{r=0}^{2\csc\theta} 1r\,dr\,d\theta$$

The value of $\iint_D 1\,dA$ is the area of the square (4 square units).

Since the outer (theta) integrals are between constant limits, we need only scale the inner (r) integrals.

Let:

* $i1 = \int_{\theta=0}^{\pi/4}\int_{r=0}^{2\sec\theta}1r\,dr\,d\theta$

* $i2 = \int_{\theta=\pi/4}^{\pi/2}\int_{r=0}^{2\csc\theta} 1r\,dr\,d\theta$

To scale the inner $i1$ integral, make the substitutions:
* $r\rightarrow 2r\sec\theta$
* $dr\rightarrow 2\sec\theta\,dr$

To scale the inner $i2$ integral, make the substitutions:
* $r\rightarrow 2r\csc\theta$
* $dr\rightarrow 2\csc\theta\,d\theta$

Doing this results in $$\int_D 1\,dA = \int_{\theta=0}^{\pi/4}\int_{r=0}^{1}4r\sec^2\theta\,dr\,d\theta +
\int_{\theta=\pi/4}^{\pi/2}\int_{r=0}^{1} 4r\csc^2\theta\,dr\,d\theta$$

In [18]:
i1 = hcubature(x->4x[1]*(sec(x[2]))^2, [0,0], [1,pi/4]);
i2 = hcubature(x->4x[1]*(csc(x[2]))^2, [0,pi/4], [1,pi/2]);

In [17]:
i1

(2.0000000000048894, 1.609450261952361e-8)

In [19]:
i2

(2.0000000000048894, 1.6094502536256883e-8)

In [20]:
i1[1]+i2[1]

4.000000000009779